||CRIM|ZN|INDUS|CHAS|NOX|RM|AGE|DIS|RAD|TAX|PTRATIO|B|LSTAT|MEDV|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|0|0.00632|18.0|2.31|0|0.538|6.575|65.2|4.0900|1|296.0|15.3|396.9|4.98|24.0|
|1|0.02731|0.0|7.07|0|0.469|6.421|78.9|4.9671|2|242.0|17.8|396.9|9.14|21.6|

In [1]:
"-|" * 14

'-|-|-|-|-|-|-|-|-|-|-|-|-|-|'